# Analyse der potentiellen Solarenergie in der Schweiz
Dieses Notebook analysiert die potentiellen Solarenergiepotenziale in der Schweiz basierend auf Dach- und Fassadenflächen.

## Daten laden, aufbereiten und anreichern
In diesem Abschnitt werden die Daten geladen, gefiltert und für die Analyse vorbereitet.

In [ ]:
# Bibliotheken importieren
import pandas as pd

# Pfad zur JSON-Datei mit den Solarenergiepotenzialen
file_path_data = '../data/potentielle_solarenergie/Solarenergiepotenziale_Gemeinden_Daecher_und_Fassaden.json'

# Daten laden
data = pd.read_json(file_path_data)

# Daten in ein DataFrame umwandeln
df = pd.DataFrame(data)

# Daten nach Country = 'CH' filtern
df = df[df['Country'] == 'CH']

# Nur relevante Spalten auswählen
df = df.rename(columns={
    'Scenario1_RoofsOnly_PotentialSolarElectricity_GWh': 'RoofsOnly_GWh',
    'Scenario3_RoofsFacades_PotentialSolarElectricity_GWh': 'RoofsFacades_GWh'
})[['Canton', 'RoofsOnly_GWh', 'RoofsFacades_GWh']]

# Fassaden-Werte bereinigen (Fassaden - Dächer)
df['FacadesOnly_GWh'] = df["RoofsFacades_GWh"] - df["RoofsOnly_GWh"]
# Spalten umbenennen

# Erste Zeilen anzeigen
df.head()

## Balkendiagramm erstellen: Potenzielle Solarenergie in der Schweiz
Das Diagramm visualisiert die potentiellen Solarenergiepotenziale nach Kanton.

In [ ]:
# Plotly für die Visualisierung importieren
import plotly.graph_objects as go

# Daten für das gestapelte Balkendiagramm vorbereiten
stacked_data = df.groupby("Canton")[["RoofsOnly_GWh", "FacadesOnly_GWh", "RoofsFacades_GWh"]].sum()

# Sortieren der Daten in absteigender Reihenfolge
stacked_data = stacked_data.sort_values(by="RoofsFacades_GWh", ascending=False)

# Farben für die Top 4 Kantone definieren
colors_roofs = ['rgb(0, 0, 0)' if i < 4 else 'rgb(169, 169, 169)' for i in range(len(stacked_data))]
colors_facades = ['rgb(128, 128, 128)' if i < 4 else 'rgb(211, 211, 211)' for i in range(len(stacked_data))]

# Erstellen des gestapelten Balkendiagramms
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Dächer',
    x=stacked_data.index,
    y=stacked_data["RoofsOnly_GWh"],
    marker_color=colors_roofs,
    text=(stacked_data["RoofsOnly_GWh"] / 1000).round(1).astype(str) + 'k',
    textposition='inside'
))
fig.add_trace(go.Bar(
    name='Fassaden',
    x=stacked_data.index,
    y=stacked_data["FacadesOnly_GWh"],
    marker_color=colors_facades,
    text=(stacked_data["FacadesOnly_GWh"] / 1000).round(1).astype(str) + 'k',
    textposition='inside'
))

# Layout anpassen
fig.update_layout(barmode='stack',
                  title='Potenzielle Solarenergie in der Schweiz nach Kanton (Stand März 2025)',
                  xaxis_title='Kanton',
                  yaxis_title='Potentielle Solarenergie (GWh)',
                  height=500,
                  template='simple_white',
                  legend_title_text='Flächenkategorie')

# Diagramm als HTML-Datei speichern
fig.write_html("../docs/assets/diagramme/pot_solarenergie_pro_kanton.html")

# Diagramm anzeigen
fig.show()

In [ ]:
# Importieren der benötigten Bibliotheken
import pandas as pd
import plotly.express as px

# Strom-Endverbrauch laden
strom_df = pd.read_csv('../data/verbrauch/Strom_Endverbrauch_Schweiz.csv', sep=';')
strom_df.columns = [col.replace(' in GWh', '') for col in strom_df.columns]
strom_endverbrauch = strom_df[strom_df['Jahr'] == strom_df['Jahr'].max()].iloc[:, 1:].sum(axis=1).values[0]

# DataFrame für Vergleich (gestapelt: Dächer und Fassaden)
pot_solar_roofs = df['RoofsOnly_GWh'].sum()
pot_solar_facades = df['FacadesOnly_GWh'].sum()

vergleich_df = pd.DataFrame({
    "Kategorie": ["Dächer", "Fassaden", "Strom-Endverbrauch"],
    "Gruppe": ["Potentielle Solarenergie", "Potentielle Solarenergie", "Strom-Endverbrauch"],
    "GWh": [pot_solar_roofs, pot_solar_facades, strom_endverbrauch]
})

# Stromproduktion aktuell laden
file_path_energy_data = '../data/energy_sources/elektrizitaetbilanz_jahreswerte.csv'
energy_data = pd.read_csv(file_path_energy_data, delimiter=';')
columns_to_keep = ['Jahr'] + [col for col in energy_data.columns if col.startswith('Erzeugung_')]
energy_data = energy_data[columns_to_keep]
energy_data['Wasserkraft'] = (energy_data['Erzeugung_laufwerk_GWh'] + energy_data['Erzeugung_speicherwerk_GWh'])
energy_data['Andere_erneuerbare_Energien_GWh'] = (energy_data['Erzeugung_holz_GWh'] + energy_data['Erzeugung_biogas_GWh'] + energy_data['Erzeugung_wind_GWh'])
energy_data = energy_data[energy_data['Jahr'] == 2024]
energy_data.rename(columns={
    'Erzeugung_kernkraftwerk_GWh': 'Kernkraft',
    'Erzeugung_andere_total_GWh': 'Thermische Kraft',
    'Erzeugung_photovoltaik_GWh': 'Photovoltaik',
    'Andere_erneuerbare_Energien_GWh': 'Andere erneuerbare Energien'
}, inplace=True)

# Werte für die wichtigsten Stromquellen extrahieren
strom_wasserkraft = energy_data['Wasserkraft'].values[0]
strom_kernkraft = energy_data['Kernkraft'].values[0]
strom_photovoltaik = energy_data['Photovoltaik'].values[0]
strom_andere_erneuerbare = energy_data['Andere erneuerbare Energien'].values[0]
strom_thermisch = energy_data['Thermische Kraft'].values[0]

# Diese Werte als weitere Bars in den Vergleich aufnehmen
vergleich_df = pd.concat([
    vergleich_df,
    pd.DataFrame({
        "Kategorie": ["Wasserkraft", "Kernkraft", "Photovoltaik", "Andere erneuerbare Energien", "Thermische Kraft"],
        "Gruppe": ["Aktuelle Stromproduktion"] * 5,
        "GWh": [strom_wasserkraft, strom_kernkraft, strom_photovoltaik, strom_andere_erneuerbare, strom_thermisch]
    })
], ignore_index=True)

vergleich_df["Text"] = vergleich_df["GWh"].apply(lambda x: f"{x:,.0f}".replace(",", "'") + " GWh")

# Text für Balken: Kategorie oberhalb, darunter Wert (bei Strom-Endverbrauch nur Wert)
def make_bar_text(row):
    value = f"{row['GWh']:,.0f}".replace(",", "'") + " GWh"
    if row['Kategorie'] == "Strom-Endverbrauch":
        return value
    else:
        return f"{row['Kategorie']}<br>{value}"

vergleich_df["Text"] = vergleich_df.apply(make_bar_text, axis=1)

import plotly.express as px
fig_vergleich = px.bar(
    vergleich_df,
    x="Gruppe",
    y="GWh",
    color="Kategorie",
    text="Text",
    color_discrete_map={
        "Dächer": "#000000",
        "Fassaden": "#A7A7A7",
        "Strom-Endverbrauch": "#636efa"
    },
    title="Vergleich: Potentielle Solarenergie (2025) vs. Strom-Endverbrauch (2024) vs. aktuelle Stromproduktion (2024)",
)
fig_vergleich.update_traces(
    textposition='inside',
    textfont_size=16,
    insidetextanchor='middle',
    showlegend=False
)
fig_vergleich.update_layout(
    barmode="stack",
    showlegend=False,
    yaxis_title="GWh",
    xaxis_title="",
    font=dict(family="Arial", size=16),
    plot_bgcolor="white",
    xaxis=dict(showline=True, linewidth=1, linecolor='black', showgrid=False, zeroline=False),
    yaxis=dict(showline=True, linewidth=1, linecolor='black', showgrid=True, zeroline=False, gridcolor="#E0E0E0"),
    margin=dict(l=40, r=20, t=60, b=40)
)

fig_vergleich.show()
fig_vergleich.write_html("../docs/assets/diagramme/pot_solarenergie_vs_strom_endverbrauch.html")